In [20]:
import pandas as pd
import numpy as np
import ast
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import label_binarize
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve, roc_auc_score
from collections import defaultdict
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix, roc_auc_score, precision_recall_curve, auc

### Read the CSV

In [3]:
train = pd.read_csv("/home/research/svea/ml_project/densenet_train_embeddings.csv", quotechar='"', on_bad_lines='skip')

print(train.columns)
train.head()

Index(['path_to_image', 'path_to_dcm', 'age', 'sex', 'race', 'insurance_type',
       'No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
       'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia',
       'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
       'Fracture', 'Support Devices', 'embeddings'],
      dtype='object')


,path_to_image,path_to_dcm,age,sex,race,insurance_type,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Opacity,...,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices,embeddings
0,train/patient02424/study8/view1_frontal.jpg,train/patient02424/study8/view1_frontal.dcm,52.0,0,0,1,0,0,0,0,...,0,0,0,1,0,1,0,0,1,"[0.0036964279133826494, 0.026995880529284477, ..."
1,train/patient08517/study10/view1_frontal.jpg,train/patient08517/study10/view1_frontal.dcm,53.0,0,1,2,0,0,1,1,...,1,0,0,0,0,0,0,0,1,"[0.0035102381370961666, 0.11090341955423355, 0..."
2,train/patient03164/study14/view1_frontal.jpg,train/patient03164/study14/view1_frontal.dcm,80.0,0,0,1,0,0,0,1,...,0,0,0,0,0,1,0,0,1,"[0.005892541725188494, 0.1605752557516098, 0.1..."
3,train/patient28576/study3/view1_frontal.jpg,train/patient28576/study3/view1_frontal.dcm,63.0,1,0,2,1,0,0,0,...,0,0,0,0,0,0,0,0,0,"[0.0012315319618210196, 0.013657770119607449, ..."
4,train/patient35066/study5/view1_frontal.jpg,train/patient35066/study5/view1_frontal.dcm,59.0,1,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,1,"[0.0032588730100542307, 0.02126752771437168, 0..."


### Convert embeddings from str to list (a bit long for large data sets)

In [4]:
train['embeddings'] = train['embeddings'].apply(ast.literal_eval)

### Remove columns

In [5]:
train = train.drop(columns=['path_to_image', 'path_to_dcm',
       'No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
       'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia',
       'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
       'Fracture', 'Support Devices'])


### Remove rows that were not processed (embeddings = 0)

In [6]:
initial_size = train.shape[0] 

train = train[train['embeddings'].apply(type) == list]

final_size = train.shape[0] 

print(f'Number of train removed rows = {initial_size - final_size}')

Number of train removed rows = 67


### Convert age to binary to study bias

In [7]:
a = 70
train['age'] = (train['age'] >= a).astype(int)

### Train test

In [8]:
subgroups = ['age', 'sex', 'race', 'insurance_type']

train_embeddings = pd.DataFrame(train['embeddings'].tolist())
y_train = train[subgroups]

x_train = pd.concat([train.reset_index(), train_embeddings], axis=1)
x_train.drop(columns=["embeddings"] + subgroups, inplace=True)

In [9]:
x_train.drop(columns=["index"], inplace=True)

In [10]:
y_sex = train['sex']
y_race = train['race']
y_age = train['age']
y_insurance = train['insurance_type']


### Sex

In [11]:
X_train, X_test, y_train_sex, y_test_sex = train_test_split(x_train, y_sex, test_size=0.2, random_state=42)

In [12]:
clf_sex = clf = RandomForestClassifier(
    n_estimators=100,       # Use 100 trees
    max_depth=10,            # Set maximum depth of each tree to 10
    min_samples_split=5,     # Minimum number of samples required to split an internal node
    min_samples_leaf=2,      # Minimum number of samples required to be at a leaf node
    max_features='sqrt',     # Number of features to consider when looking for the best split: sqrt(total_features)
    bootstrap=True,          # Use bootstrapping
    random_state=42          # Set a random state for reproducibility
)
clf_sex.fit(X_train, y_train_sex)

RandomForestClassifier(max_depth=10, min_samples_leaf=2, min_samples_split=5,
                       random_state=42)

In [13]:
y_scores_sex = clf_sex.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class
fpr, tpr, thresholds = roc_curve(y_test_sex, y_scores_sex)
auc_score = roc_auc_score(y_test_sex, y_scores_sex)
print(f"AUC Score: {auc_score}")

AUC Score: 0.8554639829998935


In [14]:
importances_sex = clf_sex.feature_importances_
# Sort and display the top N important features
indices = np.argsort(importances_sex)[::-1]
print("Top N important embeddings for sex prediction:")
for i in indices[:10]:  # top 10 features
    print(f"Embedding {i} importance: {importances_sex[i]}")



Top N important embeddings for sex prediction:
Embedding 634 importance: 0.031987947873739306
Embedding 611 importance: 0.021207261477278846
Embedding 13 importance: 0.01490744643213179
Embedding 632 importance: 0.014693097374506048
Embedding 630 importance: 0.011383139503288025
Embedding 371 importance: 0.01070354966956921
Embedding 105 importance: 0.01065773191272925
Embedding 108 importance: 0.010411652089956078
Embedding 744 importance: 0.009963929265807272
Embedding 312 importance: 0.009950422164634367


In [16]:
X_train, X_test, y_train_race, y_test_race = train_test_split(x_train, y_race, test_size=0.2, random_state=42)

clf_race = clf = RandomForestClassifier(
    n_estimators=100,       # Use 100 trees
    max_depth=10,            # Set maximum depth of each tree to 10
    min_samples_split=5,     # Minimum number of samples required to split an internal node
    min_samples_leaf=2,      # Minimum number of samples required to be at a leaf node
    max_features='sqrt',     # Number of features to consider when looking for the best split: sqrt(total_features)
    bootstrap=True,          # Use bootstrapping
    random_state=42          # Set a random state for reproducibility
)
clf_race.fit(X_train, y_train_race)

# y_scores_race = clf_sex.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class
# fpr, tpr, thresholds = roc_curve(y_test_race, y_scores_race)
# auc_score = roc_auc_score(y_test_race, y_scores_race)
# print(f"AUC Score: {auc_score}")


importances_race = clf_race.feature_importances_
# Sort and display the top N important features
indices = np.argsort(importances_race)[::-1]
print("Top N important embeddings for race prediction:")
for i in indices[:10]:  # top 10 features
    print(f"Embedding {i} importance: {importances_race[i]}")



Top N important embeddings for race prediction:
Embedding 611 importance: 0.010535659145837921
Embedding 304 importance: 0.010429737791338076
Embedding 172 importance: 0.009874259214875268
Embedding 634 importance: 0.009868407016808554
Embedding 316 importance: 0.008827436934266684
Embedding 293 importance: 0.008707826371391399
Embedding 312 importance: 0.008065844364274178
Embedding 435 importance: 0.007560679944286893
Embedding 238 importance: 0.007284878064704216
Embedding 525 importance: 0.006478203061417899


In [21]:
#Since roc curve and auc_score are only defined for binary data we are using Micro Averaging

y_test_race_binarized = label_binarize(y_test_race, classes=np.unique(y_test_race))
y_scores_race = clf_race.predict_proba(X_test)
micro_auc = roc_auc_score(y_test_race_binarized, y_scores_race, average="micro")
print(f"Micro-Averaged AUC Score: {micro_auc}")

Micro-Averaged AUC Score: 0.8971997873042445


In [18]:
X_train, X_test, y_train_age, y_test_age = train_test_split(x_train, y_age, test_size=0.2, random_state=42)

clf_age = clf = RandomForestClassifier(
    n_estimators=100,       # Use 100 trees
    max_depth=10,            # Set maximum depth of each tree to 10
    min_samples_split=5,     # Minimum number of samples required to split an internal node
    min_samples_leaf=2,      # Minimum number of samples required to be at a leaf node
    max_features='sqrt',     # Number of features to consider when looking for the best split: sqrt(total_features)
    bootstrap=True,          # Use bootstrapping
    random_state=42          # Set a random state for reproducibility
)
clf_age.fit(X_train, y_train_age)

# # Predict probabilities for the positive class
# y_scores_age = clf_age.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class
# fpr, tpr, thresholds = roc_curve(y_test_age, y_scores_age)
# auc_score = roc_auc_score(y_test_age, y_scores_age)
# print(f"AUC Score: {auc_score}")

importances_age = clf_age.feature_importances_
# Sort and display the top N important features
indices = np.argsort(importances_age)[::-1]
print("Top N important embeddings for age prediction:")
for i in indices[:10]:  # top 10 features
    print(f"Embedding {i} importance: {importances_age[i]}")



Top N important embeddings for age prediction:
Embedding 885 importance: 0.05129458479605044
Embedding 892 importance: 0.030763301199778582
Embedding 867 importance: 0.030010819063191637
Embedding 973 importance: 0.01822174921950952
Embedding 508 importance: 0.016030197645587413
Embedding 889 importance: 0.0155482992608116
Embedding 441 importance: 0.01361207199308293
Embedding 160 importance: 0.012683934840116148
Embedding 416 importance: 0.011126091419768232
Embedding 886 importance: 0.009655212483585831


In [19]:
y_scores_age = clf_age.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class
fpr, tpr, thresholds = roc_curve(y_test_age, y_scores_age)
auc_score = roc_auc_score(y_test_age, y_scores_age)
print(f"AUC Score: {auc_score}")

AUC Score: 0.7780283514261095


In [22]:
X_train, X_test, y_train_insurance, y_test_insurance = train_test_split(x_train, y_insurance, test_size=0.2, random_state=42)

clf_insurance = clf = RandomForestClassifier(
    n_estimators=100,       # Use 100 trees
    max_depth=10,            # Set maximum depth of each tree to 10
    min_samples_split=5,     # Minimum number of samples required to split an internal node
    min_samples_leaf=2,      # Minimum number of samples required to be at a leaf node
    max_features='sqrt',     # Number of features to consider when looking for the best split: sqrt(total_features)
    bootstrap=True,          # Use bootstrapping
    random_state=42          # Set a random state for reproducibility
)
clf_insurance.fit(X_train, y_train_insurance)

# y_scores_insurance = clf_sex.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class
# fpr, tpr, thresholds = roc_curve(y_test_insurance, y_scores_insurance)
# auc_score = roc_auc_score(y_test_insurance, y_scores_insurance)
# print(f"AUC Score: {auc_score}")


importances_insurance = clf_insurance.feature_importances_
# Sort and display the top N important features
indices = np.argsort(importances_insurance)[::-1]
print("Top N important embeddings for age prediction:")
for i in indices[:10]:  # top 10 features
    print(f"Embedding {i} importance: {importances_insurance[i]}")



Top N important embeddings for age prediction:
Embedding 885 importance: 0.026993045166427733
Embedding 867 importance: 0.018186180849187886
Embedding 508 importance: 0.0170813126680816
Embedding 973 importance: 0.013781295683683581
Embedding 892 importance: 0.0121509465025808
Embedding 441 importance: 0.009716967512262066
Embedding 160 importance: 0.009503985605748137
Embedding 889 importance: 0.009358132040096272
Embedding 795 importance: 0.0075231834246995855
Embedding 940 importance: 0.007106200044237633


In [23]:
#Since roc curve and auc_score are only defined for binary data we are using Micro Averaging

y_test_insurance_binarized = label_binarize(y_test_insurance, classes=np.unique(y_test_insurance))
y_scores_insurance = clf_insurance.predict_proba(X_test)
micro_auc = roc_auc_score(y_test_insurance_binarized, y_scores_insurance, average="micro")
print(f"Micro-Averaged AUC Score: {micro_auc}")

Micro-Averaged AUC Score: 0.8361155565741922
